<a href="https://colab.research.google.com/github/ericjin92/ej/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.2 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage,
    ConfirmTemplate,
    MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        action = event.message.text
        if action == 'confirm':
             reply = TemplateMessage(
                 alt_text="這是確認視窗",
                 template=ConfirmTemplate(
                     text="你喜歡看青春之箱嗎？",
                     actions=[
                        MessageAction(label="是", text="我超愛"),
                        MessageAction(label="否", text="其實我非常喜歡 但我要說不")
                         ]
                   ))

        elif action == 'carousel':
          carousel_template= CarouselTemplate(
            columns=[
                CarouselColumn(
                    thumbnail_image_url='https://images.chinatimes.com/newsphoto/2020-01-09/1024/BBC400_P_03_02.jpg',
                    title='夏日大作戰',
                    text='細田守導演的日本科幻暨浪漫電影。',
                    actions=[
                        URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0'),
                        MessageAction(label='投票', text='我投夏日大作戰一票')
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url='https://i.imgur.com/M9wqbiT.jpeg',
                    title='青春之箱',
                    text='日本漫畫家三浦糀創作的少年漫畫。',
                    actions=[
                        URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E9%9D%92%E6%98%A5%E4%B9%8B%E7%AE%B1'),
                        MessageAction(label='投票', text='我投青春之箱一票')
                    ]
                )
            ]
        )
          reply = TemplateMessage(
                 alt_text="這是輪播視窗",
                 template=carousel_template)

        else:
          reply = TextMessage(text = 'Thanks !')
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )

ngrok_start()
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1428-35-229-170-182.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 12:11:21] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 12:11:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 12:11:34] "POST /callback HTTP/1.1" 200 -
